In [1]:
import napari, os, glob
import numpy as np
from tqdm.notebook import tqdm
import pandas as pd
from skimage.io import imread, imsave
from skimage.transform import resize
import matplotlib.pyplot as plt
import pyclesperanto_prototype as cle

%matplotlib notebook

# Parameters

Define the path to the image folder and the ZYX pixel size (anisotropic)

In [5]:
img_folder = os.path.join('Y:',os.sep,'/Kerim_Anlas/gastruloid_imaging/tracking_preprocess/preprocess/20211110_ESLmix_48h_firstTex_probnochi\Pos3_best_reg_g','mKO2_reg')
scale = (2,0.3467,0.3467)


# Find all tif files in the folder

In [6]:
flist = glob.glob(os.path.join(img_folder,'*.tif'))
print('Found %d images.'%len(flist))

Found 200 images.


# Optional: visualize one processed image with napari

In [7]:
viewer = napari.Viewer()
napari.run()

print('Load sample image')
img0 = imread(flist[50])
print('Tophat')
img1 = cle.top_hat_box(img0, np.zeros(img0.shape), radius_x=25.0, radius_y=25.0, radius_z=6.0)
print('Gaussian blur')
img2 = cle.gaussian_blur(img1, sigma_x=2, sigma_y=2, sigma_z=0.2)

print('Convert to int')
img1 = img1.astype(np.uint16)
img2 = img2.astype(np.uint16)

viewer.add_image(img0, scale=scale, name='gastruloid_original', contrast_limits=(0, np.max(img0)/2))
viewer.add_image(img1, scale=scale, name='gastruloid_tophat', contrast_limits=(0, np.max(img1)/2))
viewer.add_image(img2, scale=scale, name='gastruloid_tophat_gauss', contrast_limits=(0, np.max(img2)/2))

Load sample image
Tophat
Gaussian blur
Convert to int


<Image layer 'gastruloid_tophat_gauss' at 0x210ecd6ea60>

# Process all images and save them

In [5]:
tophat_folder = os.path.join(os.path.split(img_folder)[0],'mKO2_reg_tophat')
gauss_folder = os.path.join(os.path.split(img_folder)[0],'mKO2_reg_tophat_gauss')

if not os.path.exists(tophat_folder):
    os.mkdir(tophat_folder)
if not os.path.exists(gauss_folder):
    os.mkdir(gauss_folder)

import time
i=0
for f in tqdm(flist):
    start = time.time()
    
    img0 = imread(f)
    load_time = time.time()
    
    img1 = cle.top_hat_box(img0, np.zeros(img0.shape), radius_z=6.0, radius_x=25.0, radius_y=25.0)
    tophat_time = time.time()

    img2 = cle.gaussian_blur(img1, sigma_z=0.2, sigma_x=2, sigma_y=2)
    gauss_time = time.time()
    
    img1 = img1.astype(np.uint16)
    img2 = img2.astype(np.uint16)
    
    imsave(os.path.join(tophat_folder,'t%04d.tif'%i), 
           cle.pull(img1), 
           check_contrast=False)
    save_tophat_time = time.time()

    imsave(os.path.join(gauss_folder,'t%04d.tif'%i), 
           cle.pull(img2), 
           check_contrast=False)
    save_gauss_time = time.time()

    print('TP %d: %.4f'%(i, load_time-start), 
          '%.4f'%(tophat_time-load_time), 
          '%.4f'%(gauss_time-tophat_time),
          '%.4f'%(save_tophat_time-gauss_time),
          '%.4f'%(save_gauss_time-save_tophat_time))
    
    i+=1
    

  0%|          | 0/200 [00:00<?, ?it/s]

TP 0: 1.9658 0.9089 0.0382 6.7660 1.8121
TP 1: 2.3774 0.8904 0.0469 6.9093 1.8247
TP 2: 2.2161 0.8571 0.0623 6.5250 1.8082
TP 3: 2.0925 0.8415 0.0780 6.5381 1.8020
TP 4: 2.3118 0.8571 0.0779 6.5450 1.7899
TP 5: 2.2021 0.8415 0.0779 6.5065 1.7949
TP 6: 1.8883 0.8666 0.0736 6.5985 1.7937
TP 7: 2.1551 0.8415 0.0779 6.5155 1.7962
TP 8: 1.9548 0.8726 0.0623 6.5202 1.8033
TP 9: 1.8451 0.9332 0.0696 6.5321 1.7921
TP 10: 1.8589 0.8571 0.0779 6.5159 1.7925
TP 11: 2.2598 0.8571 0.0779 6.5165 1.7921
TP 12: 2.2356 0.8571 0.0779 6.4982 1.8017
TP 13: 1.8763 0.8415 0.0779 6.6627 1.7998
TP 14: 1.8318 0.8546 0.0792 6.5834 1.7941
TP 15: 1.8353 0.8571 0.0623 6.5467 1.7971
TP 16: 1.8988 0.8570 0.0779 6.5145 1.7984
TP 17: 2.0431 0.8450 0.0779 6.5227 1.7992
TP 18: 2.1211 0.8415 0.0779 6.5044 1.8292
TP 19: 1.9040 0.8415 0.0779 6.5137 1.8066
TP 20: 2.3335 0.8419 0.0779 6.5149 1.7971
TP 21: 2.1314 0.8415 0.0779 6.5159 1.7926
TP 22: 2.3168 0.8415 0.0779 6.5172 1.8008
TP 23: 2.3063 0.8727 0.0623 6.5259 1.7986
TP

TP 194: 2.1515 0.8592 0.0781 6.5315 1.8097
TP 195: 1.8921 0.8592 0.0841 6.5404 1.7848
TP 196: 2.1658 0.8454 0.0938 6.5259 1.7965
TP 197: 1.8690 0.8592 0.0781 6.5319 1.8036
TP 198: 2.2282 0.8592 0.0781 6.5298 1.8301
TP 199: 2.2186 0.8436 0.0781 6.5667 1.7987
